In [2]:
# necessary for scraping
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
import pandas as pd
import numpy as np
import time
import random

In [3]:
# scraping url for indeed search : data science job in Vancouver BC , radius 50km 
url = 'https://www.glassdoor.ca/Job/vancouver-bc-canada-data-science-jobs-SRCH_IL.0,19_IC2278756_KO20,32.htm?sortBy=date_desc'

In [4]:
# set up web browser for scraping
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(  
    options=options
)
driver.get(url)
driver.maximize_window()

In [5]:
# expand listing
more_jobs_button = driver.find_element(By.CSS_SELECTOR , ".JobsList_buttonWrapper__haBp5")
more_jobs_button.click()

In [6]:
# close sign-in pop up
try:
    close_button = WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".CloseButton")))
    close_button.click()
except TimeoutException: pass

In [7]:
# click the show more jobs button 4 more times
for count in range(5):
    try:
        more_jobs_button = driver.find_element(By.CSS_SELECTOR , ".JobsList_buttonWrapper__haBp5")
        more_jobs_button.click()
    except NoSuchElementException: pass
    time.sleep(4)

In [8]:
# get all job posting element
job_listings = driver.find_elements(By.CSS_SELECTOR, ".JobCard_jobCardWrapper__0b8Jt")

In [12]:
# create variable to store data in job posting
posting_age = None
job_title = None
company_name = None
location = None
pay = None
job_description = None
# create list to store job posting
jobs = []
limit = 5
for job_listing in job_listings:
    # extract posting age
    try:
        posting_age_element = job_listing.find_element(By.CSS_SELECTOR, ".JobCard_listingAge__KuaxZ")
        posting_age = posting_age_element.text
    except NoSuchElementException: pass
    # click job listing
    job_listing.click()
    # wait for 10 sec for the until full job description loaded
    try: 
        job_description_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR,".JobDetails_jobDescription__6VeBn")))
    except TimeoutException: pass
    
    # access job right panel
    job = driver.find_element(By.CSS_SELECTOR, ".JobDetails_jobDetailsContainer__sS1W1")
    # click show more
    show_more_button = job.find_element(By.CSS_SELECTOR,".JobDetails_showMore__j5Z_h")
    show_more_button.click()
    time.sleep(2)
    # extract company name
    try: 
        company_name_element = job.find_element(By.CSS_SELECTOR, ".EmployerProfile_employerName__Xemli")
        company_name = company_name_element.text
    except NoSuchElementException: pass
    # extract location
    try: 
        location_element = job.find_element(By.CSS_SELECTOR,".JobDetails_location__MbnUM")
        location = location_element.text
    except NoSuchElementException: pass
    # extract pay
    try:
        pay_element = job.find_element(By.CSS_SELECTOR,".SalaryEstimate_averageEstimate__xF_7h")
        pay_period_element = job.find_element(By.CSS_SELECTOR,".SalaryEstimate_payPeriod__oBnsD")
        pay = pay_element.text + pay_period_element.text
    except NoSuchElementException: pass
    # extract job title
    try: 
        job_title_element = job.find_element(By.CSS_SELECTOR, ".JobDetails_jobTitle__Rw_gn")
        job_title = job_title_element.text
    except NoSuchElementException: pass
    # extract job description
    try:
        job_description_element = job.find_element(By.CSS_SELECTOR,".JobDetails_jobDescription__6VeBn")
        job_description = job_description_element.text
    except NoSuchElementException: pass
    # create job dictionary to store all data for 1 job posting then add to posting list
    job = {}
    job['posting_age'] = posting_age
    job['job_title'] = job_title
    job['company_name'] = company_name
    job['location'] = location
    job['pay'] = pay
    job['job_description'] = job_description
    #print(job)
    jobs.append(job)
    #limit -= 1
    #if limit == 0: break

In [13]:
# quite Chrome driver
driver.quit()

In [14]:
job_df = pd.DataFrame(jobs)
# export data table to csv file
job_df.to_csv("glassdoor_jobs.csv",index = False)
job_df

,posting_age,job_title,company_name,location,pay,job_description
0,2d,Senior Data Analyst [BHJOB13022_12997],Ignite Technical Resources.,Richmond,"$51,269/yr (est.)","On behalf of our Healthcare client, Ignite Tec..."
1,3d,Data Scientist,University of British Columbia,Vancouver,"$7,397/mo (est.)",Staff - Non Union\nJob Category\nM&P - AAPS\nJ...
2,3d,Senior Machine Learning Developer,Market Jar Media Inc,Vancouver,"$79,579/yr (est.)","One Team. One Goal.\nAt MarketJar, we are a te..."
3,4d,Senior Data Scientist,Spring Financial Inc.,Vancouver,"$125,000/yr (est.)","About us:\nSince 2014, Spring Financial has be..."
4,4d,Senior Financial Data Analyst (Adaptive Insigh...,Central 1 Credit Union,Vancouver,"$96,300/yr (est.)",Central 1 cooperatively empowers credit unions...
...,...,...,...,...,...,...
117,30d+,Data Scientist,Lightspark,Vancouver,"$96,730/yr (est.)",Who We Are\nLightspark is an innovative cleant...
118,30d+,Machine Learning Engineer,MobSquad,Vancouver,"$131,100/yr (est.)","ABOUT MOBSQUAD\nWe are a well-funded, hyper-gr..."
119,30d+,ML Engineer/Data Engineer,Railtown AI Technologies,Vancouver,"$90,353/yr (est.)",Location\nHybrid in Vancouver BC\nDepartment\n...
120,30d+,AI ML Engineer,Railtown AI Technologies,Vancouver,"$107,056/yr (est.)",Location\nHybrid in Vancouver BC\nDepartment\n...
